<a href="https://colab.research.google.com/github/younghwani/ScalpProject/blob/master/scalpVGG16Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scalp Statement Classification Project

### Library

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras import optimizers
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

import os
import shutil
import glob
import zipfile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

### Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Data file copy (from drive to colab kernel)
original_data_path = '/content/drive/MyDrive/Data/scalp.zip'
new_data_path = '/content'

shutil.copy(original_data_path, new_data_path)

In [ ]:
# Unzip
path_to_zip_file = '/content/scalp.zip'
directory_to_extract_to = '/content'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
trainDataPath = '/content/scalp/Training'
validDataPath = '/content/scalp/Validation'

trData = ImageDataGenerator()
trainData = trData.flow_from_directory(directory=trainDataPath, target_size=(224,224))
valData = ImageDataGenerator()
validData = valData.flow_from_directory(directory=validDataPath, target_size=(224,224))

### Modeling

In [ ]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)

In [ ]:
vggmodel.summary()

In [ ]:
for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

In [ ]:
H = vggmodel.layers[-2].output
predictions = Dense(7, activation="softmax")(H)
model_final = Model(inputs = vggmodel.input, outputs = predictions)

In [ ]:
model_final.compile(loss = "categorical_crossentropy",
                    optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),
                    metrics=["accuracy"])

In [ ]:
model_final.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16.h5", 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             period=1)
early = EarlyStopping(monitor='val_accuracy', 
                      min_delta=0, 
                      patience=40, 
                      verbose=1, 
                      mode='auto')

### Train

In [ ]:
model_final.fit_generator(generator= trainData, 
                          steps_per_epoch= 2, 
                          epochs= 100, 
                          validation_data= validData, 
                          validation_steps=1, 
                          callbacks=[checkpoint,early])

In [ ]:
# model_final.save_weights("/content/drive/MyDrive/Data/scalp/Model/vgg16.h5")

### Test

In [ ]:
from keras.preprocessing import image
img = image.load_img("/content/scalp/Validation/erythema/0013_A2LEBJJDE00060O_1606386266117_2_TH.jpg", target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)

from keras.models import load_model
saved_model = load_model("vgg16.h5")
output = saved_model.predict(img)

result = {0: "alopecia", 1: "dandruff", 2: "erythema", 3: "good", 4: "keratin", 5: "pustule", 6: "sebum"}
print(result[np.argmax(output)])